### Imports

In [ ]:
import logging
import time
from copy import deepcopy

from IPython.core.display import display, HTML

import os
os.chdir('H:/Mes Documents/ServiceCivil2019/schedvisu')
import sys
sys.path.append('scripts')

from main import create_logger, load_config, run_pipeline, run

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

# set the width of the notebook
display(HTML("<style>.container { width:95% !important; }</style>"))

###  Run the whole pipeline

In [ ]:
run()
logging.shutdown()

###  Run the whole pipeline for 4 years, starting by 2019

In [ ]:
date_ranges = [
    {'start': '20190101', 'end': '20191213'},
    {'start': '20180101', 'end': '20181231'},
    {'start': '20170101', 'end': '20171231'},
    {'start': '20160101', 'end': '20161231'}
]

config = load_config()
for date_range in date_ranges:
    try:
        create_logger()
        local_config = deepcopy(config)
        local_config['main']['start_date'] = date_range['start']
        local_config['main']['end_date'] = date_range['end']
        run_pipeline(local_config)

    except Exception as e:
        logging.error('Error while running workflow')
        logging.error("-"*60)
        logging.error(e, exc_info=True)
        logging.error("-"*60)

    except KeyboardInterrupt:
        logging.error('Interrupted by user')

    finally:
        logging.shutdown()
        time.sleep(1)

### Close the logging

In [ ]:
logging.shutdown()

### Run the pipeline two weeks by two weeks

In [ ]:
now_str = dt.now().strftime('%Y-%m-%d_%Hh%Mm%Ss')

In [ ]:
start_date = dt(2019, 12, 15)

prev_friday = start_date - timedelta(days=start_date.weekday() - 4)
if prev_friday > start_date: prev_friday = prev_friday - timedelta(days=7)
previous_monday_1W = prev_friday - timedelta(days=5)
previous_monday_2W = prev_friday - timedelta(days= 5 +  1 * 7 - 1)
previous_monday_4W = prev_friday - timedelta(days= 5 +  3 * 7 - 1)
previous_monday_3M = prev_friday - timedelta(days= 5 + 11 * 7 - 1)
previous_monday_6M = prev_friday - timedelta(days= 5 + 23 * 7 - 1)
previous_monday_1Y = prev_friday.replace(day=1).replace(month=1)

date_ranges = [
    { 'start': previous_monday_1W.strftime('%Y%m%d'), 'end': prev_friday.strftime('%Y%m%d') },
    { 'start': previous_monday_2W.strftime('%Y%m%d'), 'end': prev_friday.strftime('%Y%m%d') },
    { 'start': previous_monday_4W.strftime('%Y%m%d'), 'end': prev_friday.strftime('%Y%m%d') },
    { 'start': previous_monday_3M.strftime('%Y%m%d'), 'end': prev_friday.strftime('%Y%m%d') },
    { 'start': previous_monday_6M.strftime('%Y%m%d'), 'end': prev_friday.strftime('%Y%m%d') },
    { 'start': previous_monday_1Y.strftime('%Y%m%d'), 'end': prev_friday.strftime('%Y%m%d') }
]

date_ranges

In [ ]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta

start_dt = dt(2019, 9, 23)
end_dt = dt(2019, 12, 13)

date_ranges = [{
    'start': start.strftime('%Y%m%d'),
    'end': (start + timedelta(days=11)).strftime('%Y%m%d') }
    for start in pd.date_range(start_dt, end_dt, freq='2W-MON')]

config = load_config()
for date_range in date_ranges:
    try:
        create_logger()
        local_config = deepcopy(config)
        local_config['main']['start_date'] = date_range['start']
        local_config['main']['end_date'] = date_range['end']
        run_pipeline(local_config)

    except Exception as e:
        logging.error('Error while running workflow')
        logging.error("-"*60)
        logging.error(e, exc_info=True)
        logging.error("-"*60)

    except KeyboardInterrupt:
        logging.error('Interrupted by user')

    finally:
        logging.shutdown()
        time.sleep(1)